# In this code we will perform the data ingestion and manipulation that will be later used in the modular project

In [1]:
# Importing core libraries

import os 
import sys

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionManipulationConfig:
    root_dir: Path
    local_data_file: Path
    source_URL: str
    save_training_file: Path

In [3]:
from Abalone_Age_Prediction.utils.common import create_directories, read_yaml
from Abalone_Age_Prediction.constants import *
from Abalone_Age_Prediction import logger

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #the artifacts_root is the key of the dictionary created
                                                # in the yaml file and we can read this key like that instead of
                                                # ["artifacts_root"] because we used the ConfigBox in the common.py file


    def get_data_ingestion_config(self) -> DataIngestionManipulationConfig:
        config = self.config.data_ingestion_manipulation #data ingestion is the other key value of the dictionary in the config.yaml file

        create_directories([config.root_dir])

        data_ingestion_manipulation_config = DataIngestionManipulationConfig(
            root_dir=config.root_dir,
            local_data_file = config.local_data_file,
            save_training_file= config.save_training_file
        )                                     

        return data_ingestion_manipulation_config

In [4]:
import urllib.request as request

In [ ]:
class DataIngestionManipulation:
    def __init__(self, config: DataIngestionManipulationConfig):
        self.config = config

    def download_file(self):
        """
        file_path: str
        Download, if it doesn't already exists, the csv file with data, don't need a return, just to save the Data
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} is downloading with the following info: \n{headers}")
        else:
            logger.info(f"{filename} already downloaded and has {get_size(Path(self.config.local_data_file))} size")
        
        return None

    
